In [1]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from __future__ import print_function
import torch
from torch.autograd import Variable
import random
from sklearn.model_selection import train_test_split

In [2]:
from keras import backend as K
from keras.layers import Embedding,Dense
from keras.layers import LSTM, Input, merge, Lambda
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Convolution1D
from keras.models import Model
import numpy as np

Using TensorFlow backend.


In [3]:
file = "glove.6B.100d.txt"
import numpy as np
def loadGloveModel(gloveFile):
    print ("Loading Glove Model")
    
    with open(gloveFile, encoding="utf8" ) as f:
       content = f.readlines()
    model = {}
    for line in content:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print ("Done.",len(model)," words loaded!")
    return model


In [4]:
model= loadGloveModel(file)   
stemmer = PorterStemmer()
lemmatiser= WordNetLemmatizer()


Loading Glove Model
Done. 400001  words loaded!


In [5]:
import string
questions=[]
answers=[]
answer=[]
length=30
sentence_length=length
with open("sample1.txt","r") as f:
    choice=0
    for line in f:
        line=line.lower()
        sentence=line.translate(string.punctuation)
        sentence=line[2:].split()
        word_vec=[]
        for word in sentence:
            if word in model:
                word_vec.append(model[word])
            else:
                stemmed_word=stemmer.stem(word)
                lemmatised_word=lemmatiser.lemmatize(word)
                if stemmed_word in model:
                    word_vec.append(model[stemmed_word])
                elif lemmatised_word in model:
                    word_vec.append(model[lemmatised_word])        
        if choice==0:
            choice=1
            if len(word_vec)<=sentence_length:
                for vec_len in range(length-len(word_vec)):
                    word_vec.append(model["0"])
            questions.append(word_vec[:length])
            
        else:
            choice=0
            if len(word_vec)<=sentence_length:
                for vec_len in range(length-len(word_vec)):
                    word_vec.append(model["0"])
                answers.append(word_vec)
                answer.append(line)
            else:
                del questions[-1]
                
        

In [6]:
input_vec_len=100
questions=np.asarray(questions)
answers=np.asarray(answers)

In [7]:
from sklearn.utils import shuffle

In [8]:
questions,answers=shuffle(questions,answers)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(questions, answers, test_size=0.05, random_state=42)

In [10]:
bad_y_train=[y_train[random.randint(0,len(y_train)-1)] for i in range(len(y_train)) ]

In [11]:
bad_y_train=np.asarray(bad_y_train)

In [12]:
bad_y_train.shape,y_test.shape

((906, 30, 100), (48, 30, 100))

In [13]:
import keras

In [14]:
questions, good_answers, bad_answers=X_train,y_train,bad_y_train
Y = np.zeros(shape=(questions.shape[0],))

In [15]:
bad_answers.shape

(906, 30, 100)

In [16]:
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

def loss_shape(shapes):
    shape1, shape2 = shapes
    return  0

In [18]:
import tensorflow as tf
from keras import backend as k
from keras.layers import Embedding,Dense,Flatten,Activation,RepeatVector,Permute,multiply

margin = 0.2
enc_timesteps = 60
dec_timesteps = 60
hidden_dim = 15
epoch=10
maxlen=60
# initialize the question and answer shapes and datatype
question = Input(shape=(length,100))
answer = Input(shape=(length,100))
answer_good = Input(shape=(length,100))
answer_bad = Input(shape=(length,100))

q_bilstm=Bidirectional(LSTM(activation='tanh', units=hidden_dim, return_sequences=True))(question)
q_atten=attention = Dense(1, activation='tanh')(q_bilstm)
q_atten = Flatten()(q_atten)
q_atten = Activation('softmax')(q_atten)
q_atten = RepeatVector(length)(q_atten)
q_atten = Permute([2, 1])(q_atten)
q_sent_representation = Lambda(lambda xin: K.sum(xin[0]*xin[1], axis=-2), output_shape=(length,))([q_bilstm, q_atten])

a_bilstm=Bidirectional(LSTM(activation='tanh', units=hidden_dim, return_sequences=True))(answer)
a_atten=attention = Dense(1, activation='tanh')(a_bilstm)
a_atten = Flatten()(a_atten)
a_atten = Activation('softmax')(a_atten)
a_atten = RepeatVector(length)(a_atten)
a_atten = Permute([2, 1])(a_atten)
a_sent_representation = Lambda(lambda xin: K.sum(xin[0]*xin[1], axis=-2), output_shape=(length,))([a_bilstm, a_atten])




question_enc_1 = q_sent_representation

answer_enc_1 = a_sent_representation

distance = Lambda(cosine_distance, output_shape=cos_dist_output_shape)([question_enc_1, answer_enc_1])

lstm_model = Model(inputs=[question, answer], outputs=[distance])
good_similarity = lstm_model([question, answer_good])

bad_similarity = lstm_model([question, answer_bad])

loss=Lambda(lambda x: K.relu(margin - x[0] + x[1]),output_shape=lambda x: x[0])([good_similarity, bad_similarity])


# return training and prediction model
training_model = Model(inputs=[question, answer_good, answer_bad], outputs=loss, name='training_model')
training_model.compile(loss=lambda y_true, y_pred: y_pred, optimizer="rmsprop")
prediction_model = Model(inputs=[question, answer_good], outputs=good_similarity, name='prediction_model')
prediction_model.compile(loss=lambda y_true, y_pred: y_pred, optimizer="rmsprop")


 

In [24]:
for i in range(300):
    bad_answers=[y_train[random.randint(0,len(y_train)-1)] for i in range(len(y_train)) ]
    bad_answers=np.asarray(bad_answers)
    print(i)
    training_model.fit(
                [questions, good_answers, bad_answers],
                Y,
                epochs=1,
                batch_size=20,
                validation_split=0.1,
                verbose=1
            )


0
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 5s 6ms/step - loss: 0.1676 - val_loss: 0.1811
1
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 5s 6ms/step - loss: 0.1688 - val_loss: 0.1784
2
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 6s 8ms/step - loss: 0.1691 - val_loss: 0.1780
3
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 6s 7ms/step - loss: 0.1680 - val_loss: 0.1751
4
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 6s 7ms/step - loss: 0.1679 - val_loss: 0.1732
5
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 5s 7ms/step - loss: 0.1701 - val_loss: 0.1802
6
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 5s 7ms/step - loss: 0.17

815/815 [==============================] - 5s 7ms/step - loss: 0.1663 - val_loss: 0.1713
112
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 5s 6ms/step - loss: 0.1671 - val_loss: 0.1800
113
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 5s 7ms/step - loss: 0.1674 - val_loss: 0.1766
114
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 6s 7ms/step - loss: 0.1672 - val_loss: 0.1794
115
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 5s 6ms/step - loss: 0.1671 - val_loss: 0.1764
116
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 6s 7ms/step - loss: 0.1688 - val_loss: 0.1763
117
Train on 815 samples, validate on 91 samples
Epoch 1/1
815/815 [==============================] - 7s 9ms/step - loss: 0.1680 - val_loss: 0.1719
118
Train on 815 sample

KeyboardInterrupt: 

In [ ]:
acc=0
for i in range(len(X_test)):
    
#     print("################################",i)
    answers=y_test
    question=[X_test[i]]*len(X_test)

    sims = prediction_model.predict([question, answers])
    sims=list(sims)

    a=sorted(np.unique(sims),reverse=True)
    element=sims[i]
    acc+=1/(a.index(element)+1)

#     print(acc)
print("MRR",acc/len(X_test))    


In [23]:
from keras.models import load_model

prediction_model.save('Self-attention network model-c.h5')  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')